# Ejercicio 2: AES

## Apartado a

A continuación se define el cuerpo $R = \mathbb{Z}_2 [x] / (x^8 + x^4 + x^3 + x + 1)\cong GF(2^8)$, como indican las especificaciones del algoritmo.

In [1]:
R = PolynomialRing(GF(2), 'x').quotient('x^8 + x^4 + x^3 + x + 1', 'x')
R

Univariate Quotient Polynomial Ring in x over Finite Field of size 2 with modulus x^8 + x^4 + x^3 + x + 1

### Transformaciones entre las distintas formas de representar un byte

In [2]:
# Binario a hexadecimal
def bin_to_hex(binary):
    '''
    Argumentos:
        - binary: Cadena binaria de longitud 8 que representa un byte en binario
    Salidas:
        - Cadena en hexadecimal de longitud 2 que representa un byte en hexadecimal
    '''
    return hex(int(binary,2))[2:].zfill(2)

# Hexadecimal a binario
def hex_to_bin(hexadecimal):
    '''
    Argumentos:
        - hexadecimal: Cadena hexadecimal de longitud 2 que representa un byte en hexadecimal
    Salidas:
        - Cadena en binario de longitud 8 que representa un byte en binario
    '''
    return bin(int(hexadecimal, 16))[2:].zfill(8)

# Polinomio correspondiente a un binario
def bin_to_pol(binary):
    '''
    Argumentos:
        - binary: Cadena binaria de longitud 8 que representa un byte en binario
    Salidas:
        - Polinomio de GF(2^8) que representa el byte.
    '''
    binary_list_int = [int(c) for c in binary[::-1]]
    return R(binary_list_int)

# Binario correspondiente a un polinomio
def pol_to_bin(pol):
    '''
    Argumentos:
        - pol: Polinomio de GF(2^8) que representa el byte.
    Salidas:
        - Cadena en binario de longitud 8 que representa un byte en binario
    '''
    return ''.join([str(c) for c in pol.list()[::-1]])

# Hexadecimal correspondiente a un polinomio
def pol_to_hex(pol):
    '''
    Argumentos:
        - pol: Polinomio de GF(2^8) que representa el byte.
    Salidas:
        - Cadena en hexadecimal de longitud 2 que representa un byte en hexadecimal
    '''
    return bin_to_hex(pol_to_bin(pol))

# Polinomio correspondiente a un hexadecimal
def hex_to_pol(hexadecimal):
    '''
    Argumentos:
        - hexadecimal: Cadena hexadecimal de longitud 2 que representa un byte en hexadecimal
    Salidas:
        - Polinomio de GF(2^8) que representa el byte.
    '''
    return bin_to_pol(hex_to_bin(hexadecimal))

Definimos la siguiente matriz que se usa en la transformación afín en el cálculo de la S-Box.

In [3]:
M = matrix(Integers(2),8,[
        1,0,0,0,1,1,1,1,
        1,1,0,0,0,1,1,1,
        1,1,1,0,0,0,1,1,
        1,1,1,1,0,0,0,1,
        1,1,1,1,1,0,0,0,
        0,1,1,1,1,1,0,0,
        0,0,1,1,1,1,1,0,
        0,0,0,1,1,1,1,1,
    ])
M

[1 0 0 0 1 1 1 1]
[1 1 0 0 0 1 1 1]
[1 1 1 0 0 0 1 1]
[1 1 1 1 0 0 0 1]
[1 1 1 1 1 0 0 0]
[0 1 1 1 1 1 0 0]
[0 0 1 1 1 1 1 0]
[0 0 0 1 1 1 1 1]

Calculamos la inversa de dicha matriz, necesaria para calcular la transformación afín inversa.

In [4]:
# Inversa de la matriz M
M_inv = M**(-1)
M_inv

[0 0 1 0 0 1 0 1]
[1 0 0 1 0 0 1 0]
[0 1 0 0 1 0 0 1]
[1 0 1 0 0 1 0 0]
[0 1 0 1 0 0 1 0]
[0 0 1 0 1 0 0 1]
[1 0 0 1 0 1 0 0]
[0 1 0 0 1 0 1 0]

De esta forma, podemos definir siguiente función, `sbox(x,y)`, que dado un byte `xy` sea capaz de aportar la entrada correspondiente a la fila `x` y la columna `y` en la tabla de la Figura 4.9 de los apuntes.

Para ello, se ha calculado la inversa de la transformación afín (4.3) seguida de la toma del inverso multiplicativo en $GF(2^8)$.

In [8]:
def sbox(x,y):
    xy = x + y
    byte_bin = hex_to_bin(xy) # Polinomio correspondiente de xy
    c = vector(Integers(2), [1,1,0,0,0,1,1,0]) # Coeficientes del byte {63}
    b = vector(Integers(2), [int(c) for c in byte_bin[::-1]]) # Vector que representa al byte de entrada

    res_transf = M_inv*(b-c) # Inversa de la transformacion afin
    
    # Inverso en GF(2^8)
    if (res_transf == 0):
        res = '00'
    else:
        res_transf_pol = bin_to_pol(''.join([str(c) for c in res_transf[::-1]]))
        res_pol = R(res_transf_pol**(-1))
        res = pol_to_hex(res_pol)
        
    return res

x = '5'
y = '3'
sbox(x,y)

'50'

Luego la posición 53 en la tabla 4.9 viene ocupada por `50`.

## Apartado b

### Matriz de `S-Box inversa`

Obtenemos la matriz que corresponde a S-box inversa, que contiene los valores de sustitución para el byte `xy` (en formato hexadecimal).

In [ ]:
hex_digits = [hex(i)[2:] for i in range(16)]
s_box = []

for x in hex_digits:
    row = []
    
    for y in hex_digits:
        row.append(sbox(x,y)) # Calcular la entrada en S-box inversa del byte xy
    
    s_box.append(row)

# Imprimir por pantalla la matriz S-Box inversa    
for row in s_box:
    print(row)